In [1]:
!pip3 install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu111 --upgrade

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu111
     |████████████████████████████████| 750.6 MB 12 kB/s 
     |████████████████████████████████| 21.0 MB 11.5 MB/s 
     |████████████████████████████████| 2.9 MB 35.3 MB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.10.0+cu111
    Uninstalling torch-1.10.0+cu111:
      Successfully uninstalled torch-1.10.0+cu111
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.11.1+cu111
    Uninstalling torchvision-0.11.1+cu111:
      Successfully uninstalled torchvision-0.11.1+cu111
  Attempting uninstall: torchaudio
    Found existing installation: torchaudio 0.10.0+cu111
    Uninstalling torchaudio-0.10.0+cu111:
      Successfully uninstalled torchaudio-0.10.0+cu111
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchtext 0.11.

In [2]:
! nvidia-smi

Tue Apr  5 05:42:55 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   47C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
! pip install kaggle tensorboard wget
! pip install tensorboard --upgrade

  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9675 sha256=06d2fc4046fc792b9336473a7052a9dd1d8c465f4f539f92b99603d9f6b9df80
  Stored in directory: /root/.cache/pip/wheels/a1/b6/7c/0e63e34eb06634181c63adacca38b79ff8f35c37e3c13e3c02
Successfully built wget


In [4]:
! mkdir ~/.kaggle

In [5]:
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle datasets download -d gverzea/edible-wild-plants
! unzip -q /content/edible-wild-plants.zip

 99% 761M/766M [00:16<00:00, 29.2MB/s]
100% 766M/766M [00:16<00:00, 47.9MB/s]


In [6]:
%load_ext tensorboard
%tensorboard --logdir logs

<IPython.core.display.Javascript object>

In [7]:
!git clone https://github.com/alexfrst/data-augmentation-benchmark

Cloning into 'data-augmentation-benchmark'...
remote: Enumerating objects: 185, done.
remote: Counting objects: 100% (185/185), done.
remote: Compressing objects: 100% (103/103), done.
remote: Total 185 (delta 101), reused 156 (delta 72), pack-reused 0
Receiving objects: 100% (185/185), 28.51 KiB | 3.56 MiB/s, done.
Resolving deltas: 100% (101/101), done.


In [1]:
import sys

import torch
import torch.nn as nn
import torch.optim as optim

sys.path.append("data-augmentation-benchmark")

import model.model as model
from torch.optim import lr_scheduler
from model.evaluate import evaluate
from model.train import train_model
from utils.dataset import load_dataset
from utils.print_utils import Symbols
from utils.tensorboard import tb_writer
from augmentation.augmentation import transforms_list


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

print(f"Working on {device} {Symbols.OK}")

Working on cuda:0 ✓


In [3]:
results = {}
for transform, train_name in transforms_list:
    torch.cuda.empty_cache()

    train_loader, val_loader, test_loader, nb_classes = load_dataset("datasets/dataset", batch_size=24, additional_transforms=(transform,))

    convnext_small = model.load_convnext_small(nb_classes, transfer_learning=False)

    criterion = nn.CrossEntropyLoss()

    optimizer = optim.SGD(convnext_small.parameters(), lr=0.001, momentum=0.9)
    scheduler = lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.8)


    print(f"Apprentissage en fine tunning avec convnext_small augment: {train_name}")
    convnext_small.to(device)
    convnext_small.train(True)

    best_model, best_val_score =  train_model(convnext_small, train_loader, val_loader, optimizer, scheduler, criterion, evaluate, 2, device=device, tb_writer=tb_writer, training_name="convnext_small_" + train_name)

    convnext_small.train(False)
    loss, accuracy = evaluate(best_model, test_loader, device, criterion)
    results[train_name] = accuracy
    print(f"Accuracy {train_name} (test): {(100 * accuracy):.1f}")
print(results)

Loading dataset...
Nombre d'images de train : 6688
Nombre d'images de val : 525
Nombre d'images de test : 1312
Apprentissage sur 62 classes
Data loading and integrity check done ✓
Apprentissage en fine tunning avec convnext_small ColorJitter-Small


Epoch 0: 0it [00:03, ?it/s]


KeyboardInterrupt: 